# PySAM Workshop

Oct 14, 2020

dguittet

https://sam.nrel.gov/software-development-kit-sdk/pysam.html

Edit: Most recently tested with Version 4.1.0

## How to Get Started

64-bit Python 3.6-3.8 for Linux, Mac and Windows

PyPi: 
```
pip install nrel-pysam
```

Anaconda:
```
conda install -c nrel nrel-pysam
```

## Model Initialization

Each technology and financial configuration is composed of unit modules. Module documentation is in [Modules](https://nrel-pysam.readthedocs.io/en/master/Models.html) or refer to [Module Index](https://nrel-pysam.readthedocs.io/en/master/py-modindex.html). 

Each module can be imported by:

```
import PySAM.<Module name>
```

There are four ways to initialize a model:

1. new
2. default
3. from_existing
4. wrap


In [1]:
import PySAM.Pvwattsv8 as pv
import PySAM.Utilityrate5 as ur

### New

Creates an instance with empty attributes.

In [2]:
new_model = pv.new()
print(type(new_model))
new_model.export()

<class 'Pvwattsv8'>


{'SolarResource': {},
 'Lifetime': {},
 'SystemDesign': {},
 'AdjustmentFactors': {},
 'Outputs': {}}

### Default

There is a lot of data needed to run a model. Entering that data with assignment statements in PySAM can be tedious. One way to get a full set of data is to load the default setup, which are the same as in SAM. A module's default values are unique for each SAM configuration type. 

Default config names are __case insensitive__. The list of options can be found on the module's PySAM doc page or as below:

In [3]:
# list configuration options
help(pv.default)

Help on built-in function default in module PySAM.Pvwattsv8:

default(...)
    default(config) -> Pvwattsv8
    
    Load defaults for the configuration ``config``. Available configurations are:
    
                    - *"FuelCellCommercial"*
    
                    - *"FuelCellSingleOwner"*
    
                    - *"PVWattsBatteryCommercial"*
    
                    - *"PVWattsBatteryHostDeveloper"*
    
                    - *"PVWattsBatteryResidential"*
    
                    - *"PVWattsBatteryThirdParty"*
    
                    - *"PVWattsAllEquityPartnershipFlip"*
    
                    - *"PVWattsCommercial"*
    
                    - *"PVWattsCommunitySolar"*
    
                    - *"PVWattsHostDeveloper"*
    
                    - *"PVWattsLCOECalculator"*
    
                    - *"PVWattsLeveragedPartnershipFlip"*
    
                    - *"PVWattsMerchantPlant"*
    
                    - *"PVWattsNone"*
    
                    - *"PVWattsResidential"

In [4]:
default_model = pv.default("FuelCellCommercial")
default_model.export()

{'SolarResource': {'albedo': (0.2,
   0.2,
   0.2,
   0.2,
   0.2,
   0.2,
   0.2,
   0.2,
   0.2,
   0.2,
   0.2,
   0.2),
  'use_wf_albedo': 1.0},
 'Lifetime': {'analysis_period': 25.0,
  'dc_degradation': (0.5,),
  'system_use_lifetime_output': 1.0},
 'SystemDesign': {'array_type': 1.0,
  'azimuth': 180.0,
  'batt_simple_enable': 0.0,
  'bifaciality': 0.0,
  'dc_ac_ratio': 1.15,
  'en_snowloss': 0.0,
  'gcr': 0.3,
  'inv_eff': 96.0,
  'losses': 14.0757,
  'module_type': 0.0,
  'soiling': (0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0),
  'system_capacity': 215.0,
  'tilt': 20.0},
 'AdjustmentFactors': {'constant': 0.0},
 'Outputs': {}}

This is good if your situation is close to the default.

Often, however, this is not the case. When your setup is significantly different than the default,
you can enter the data using SAM, and save that data to a JSON file, which can then be loaded by PySAM.
You can then modify and use the data in PySAM as needed. This is shown in [To import a case from the SAM GUI](https://nrel-pysam.readthedocs.io/en/master/Import.html) and was a subject of last year's webinar.

### from_existing

When running more than one unit module in a sequence, data needs to get passed from one to the subsequent models. For example, a technology module's `analysis_period` and generation profile `gen`, are inputs to the utility rate calculator in order to calculate the `annual_energy_value`, the energy value in each year of the analysis period due to electricity bill savings.

`from_existing` is used create a new model that shares the underlying data with an existing model. This new model can be created with default values if a default configuration name is provided, similar to ``default``.

In [5]:
shared_model = ur.from_existing(default_model, "FuelCellCommercial")
print(default_model.Lifetime.analysis_period)
print(shared_model.Lifetime.analysis_period)

25.0
25.0


In [6]:
# change in analysis period reflected in both models
default_model.Lifetime.analysis_period = 20
print(default_model.Lifetime.analysis_period)
print(shared_model.Lifetime.analysis_period)

20.0
20.0


All variables and their values, including inputs and outputs (after model execution), are shared between models linked in this way. When the PV model below is executed, its outputs will automatically be available to the Utility rate model. __Order of execution matters.__

Simulation data is passed between unit modules when the name of a unit module's output is the same as another unit module's input, such as `gen`. __The group names can be different.__ 

In [7]:
# gen doesn't exist yet because simulation hasn't been executed
default_model.Outputs.gen

(0.0,)

In [8]:
# can't calculate utility rate value without a generation profile
try:
    shared_model.execute(0)
except Exception as e:
    print(e)

utilityrate5 execution error.
	exec fail(utilityrate5): invalid number of gen records (0): must be an integer multiple of 8760




In [9]:
# execute PV then utility rate calculations
default_model.SolarResource.solar_resource_file = "/Users/dguittet/SAM Downloaded Weather Files/weather.csv"
default_model.execute(0)
print('gen\n', default_model.Outputs.gen[0:10])

shared_model.execute(0)
print('\nannual energy value\n', shared_model.Outputs.annual_energy_value)

gen
 (0.0, 0.0, 0.0, 0.0, 0.0, -0.0, 6.389046797900428, 37.94398351785478, 67.06449907636099, 92.89776429848214)

annual energy value
 (0.0, 19406.19405690556, 19800.52382533316, 20202.44148167006, 20612.079886380227, 21029.57424662404, 21455.061069906493, 21888.678750448922, 22330.56790184979, 22780.871493923893, 23239.73432549519, 23707.302537008927, 24183.72454046737, 24669.151054164817, 25162.80184412058, 25665.452721768244, 26177.509583152998, 26699.12880537708, 27230.46804522286, 27771.6872915576, 28322.60044366972)


### wrap

Creates a model from a PySSC data structure. This allows compatibility with PySSC. 

This is used primarily during data import from SAM via JSON. This import feature was covered in the [2019 PySAM Webinar](https://sam.nrel.gov/software-development-kit-sdk/pysam.html) and is also shown in [To import a case from the SAM GUI](https://nrel-pysam.readthedocs.io/en/master/Import.html).

In [10]:
import PySAM.PySSC as pyssc

ssc = pyssc.PySSC()
pv_dat = ssc.data_create()
ssc.data_set_number(pv_dat, b'analysis_period', 10)

wrap_model = pv.wrap(pv_dat)
wrap_model.export()

{'SolarResource': {},
 'Lifetime': {'analysis_period': 10.0},
 'SystemDesign': {},
 'AdjustmentFactors': {},
 'Outputs': {}}

## Detailed PV-Battery - Commercial Owner Example

In [11]:
import PySAM.Pvsamv1 as pvsam
import PySAM.Grid as grid
import PySAM.Utilityrate5 as ur
import PySAM.Cashloan as cl
import PySAM

print(PySAM.__version__)

4.1.0


In [12]:
# get all years of weather data
import glob
import random
import PySAM.ResourceTools as tools
from PySAM.BatteryTools import battery_model_sizing

weather_folder = "/Users/dguittet/SAM Downloaded Weather Files/lexington_or/"
weather_files = glob.glob(weather_folder + "/*.csv")

# load data from file into dictionaries
weather_data = [tools.SAM_CSV_to_solar_data(f) for f in weather_files]
steps_per_year = len(weather_data[0]['year'])
print(steps_per_year)

# initialize all models
pvbatt_model = pvsam.default("PVBatteryCommercial")
grid_model = grid.from_existing(pvbatt_model, "PVBatteryCommercial")
ur_model = ur.from_existing(pvbatt_model, "PVBatteryCommercial")
cl_model = cl.from_existing(pvbatt_model, "PVBatteryCommercial")

# change simulation settings
pvbatt_model.Lifetime.analysis_period = 1
pvbatt_model.value("batt_room_temperature_celsius", (25,) * steps_per_year)
pvbatt_model.BatteryDispatch.batt_dispatch_choice = 0  # peak shaving

17520


In [13]:
def installed_cost(pv_kw, battery_kw, battery_kwh):
    return pv_kw * 700 + battery_kw * 600 + battery_kwh * 300

print("batt_kw\tbatt_kwh\tavg_npv")
for battery_kw in range(10, 100, 10):
    battery_kwh = 4 * battery_kw  # four hour battery
    battery_model_sizing(pvbatt_model, battery_kw, battery_kwh, 500)
    
    npvs = []
    for solar_resource_data in weather_data:
        
        pvbatt_model.SolarResource.solar_resource_data = solar_resource_data
        pvbatt_model.execute(1)
        
        grid_model.execute(0)
        
        ur_model.execute(0)
        
        cl_model.total_installed_cost = installed_cost(pvbatt_model.SystemDesign.system_capacity, battery_kw, battery_kwh)
        cl_model.execute(0)
        
        npvs.append(cl_model.Outputs.npv)
        
    avg_npv = sum(npvs) / len(npvs)
    print("{}\t{}\t{}".format(battery_kw, battery_kwh, avg_npv))

batt_kw	batt_kwh	avg_npv
10	40	-131785.9623201237
20	80	-128677.60802359802
30	120	-126139.5470983437
40	160	-124008.21375974927
